In [ ]:
# Imports
import os
from datetime import datetime, timedelta
import numpy as np
import pandas as pd
import random
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tqdm.notebook import tqdm
import json

# Constants
DATA_DIR = "/content/drive/MyDrive/Leren_en_Beslissen/data lagersmit/processed data/normalized/"

INPUT_COLUMNS = pd.Index(['averageLeakageRolling', 'averageLeakageCum', 'allowedLeakage', 'AI1', 'AI2', 'AI3', 'TimeElapsed'])
OUTPUT_COLUMNS = pd.Index(['averageLeakageRolling'])
INPUT = 2016
OUTPUT = 2016

TOTAL = INPUT + OUTPUT
INPUT_WIDTH = len(INPUT_COLUMNS)
OUTPUT_WIDTH = len(OUTPUT_COLUMNS)
INPUT_INDICES = [INPUT_COLUMNS.get_loc(output) for output in OUTPUT_COLUMNS]

BATCH_SIZE = 32
N_TRAIN_FILES = 100
FILE_LOOPS = 3

sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True))

Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7



In [ ]:
timedelta(days=91)/timedelta(minutes=5)

26208.0

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Data

In [ ]:
all_files = [f for f in os.listdir(DATA_DIR) if os.path.isfile(os.path.join(DATA_DIR, f))]
random.shuffle(all_files)

train_list = []
val_list = []
test_list = []

for file in all_files:
  df = pd.read_feather(DATA_DIR + file)[INPUT_COLUMNS]

  if len(df.index) >= INPUT + OUTPUT:
    if len(test_list) < 0.1*len(train_list):
      test_list.append((file, df))
    elif len(val_list) < 0.2*len(train_list):
      val_list.append((file, df))
    else:
      train_list.append((file, df))
  else:
    print(f'file {file} is to small.')
  
  if len(train_list) >= N_TRAIN_FILES and len(val_list) >= 1 and len(test_list) >= 1:
    break

print(f'\ntraining files: {len(train_list)}\t validation files: {len(val_list)}\t test files: {len(test_list)}')

def split_window(features):
  inputs = features[:, slice(0, INPUT), :]
  labels = tf.stack([features[:, slice(INPUT, None), i] for i in INPUT_INDICES], axis=-1)
  # if self.label_columns is not None:
  #   labels = tf.stack(
  #       [labels[:, :, self.column_indices[name]] for name in self.label_columns],
  #       axis=-1)
  inputs.set_shape([None, INPUT, None])
  labels.set_shape([None, OUTPUT, None])

  return inputs, labels

def to_window(data):
  data = np.array(data, dtype=np.float32)
  window = keras.utils.timeseries_dataset_from_array(
      data=data,
      targets=None,
      sequence_length=TOTAL,
      batch_size=BATCH_SIZE)
  
  splitted_window = window.map(split_window)

  return splitted_window

file copies-65-b_Finnborg - 9419321 - 2 (18 days 8 hours).feather is to small.
file copies-65-a_Vessels_Fraserborg - 9419319_X25077A - 1 (6 days 8 hours).feather is to small.
file copies-60-b_MEUSE RIVER - 9866988_X22835A - 7 (5 days 1 hours).feather is to small.
file copies-70-a_Vessels_Star Paola - 9497579 - 1 (9 days 23 hours).feather is to small.
file copies-60-b_Minerva Iris - 9285861_X22980A - 2 (13 days 1 hours).feather is to small.
file copies-65-a_Vessels_MEUSE RIVER - 9866988_X22835A - 1 (7 days 2 hours).feather is to small.
file copies-60-b_MEUSE RIVER - 9866988_X22835A - 6 (11 days 2 hours).feather is to small.
file copies-65-b_Frisian Ocean - 9385922_X9384A - 1 (5 days 21 hours).feather is to small.
file copies-60-b_Navig8 Ammolite - 9727534_X24368A - 2 (1 days 23 hours).feather is to small.
file copies-60-a_La Datcha_SB - 1 (10 days 6 hours).feather is to small.
file copies-60-a_Vessels_Fraserborg - 9419319_X25077A - 2 (7 days 9 hours).feather is to small.
file copies-60-

# Model

In [ ]:
input = keras.Input(shape=(INPUT, INPUT_WIDTH))
lstm = layers.LSTM(32)(input)
dense = layers.Dense(OUTPUT * OUTPUT_WIDTH)(lstm)
output = layers.Reshape((OUTPUT, OUTPUT_WIDTH))(dense)

model = keras.Model(input, output)
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 2016, 7)]         0         
                                                                 
 lstm_1 (LSTM)               (None, 32)                5120      
                                                                 
 dense_1 (Dense)             (None, 2016)              66528     
                                                                 
 reshape_1 (Reshape)         (None, 2016, 1)           0         
                                                                 
Total params: 71,648
Trainable params: 71,648
Non-trainable params: 0
_________________________________________________________________


In [ ]:
window = to_window(train_list[0][1])

print(window.element_spec)

batch = next(iter(window))

print("input shape:", batch[0].shape)
print("output shape:", batch[1].shape)
print("result shape:", model(batch[0]).shape)


(TensorSpec(shape=(None, 2016, 7), dtype=tf.float32, name=None), TensorSpec(shape=(None, 2016, 1), dtype=tf.float32, name=None))
input shape: (32, 2016, 7)
output shape: (32, 2016, 1)
result shape: (32, 2016, 1)


In [ ]:
model.compile(
    loss=tf.losses.MeanSquaredError(),
    optimizer=tf.optimizers.Adam(clipvalue = 0.1),
    metrics=[tf.metrics.MeanAbsoluteError()],
    run_eagerly=True)

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                    patience=2,
                                                    mode='min')

for _ in tqdm(range(FILE_LOOPS), desc='iteration'):
  random.shuffle(train_list)

  for train_file, train_df in tqdm(train_list, desc='file'):
    val_file, val_df = random.choice(val_list)

    train_window = to_window(train_df)
    val_window = to_window(val_df)

    history = model.fit(
      train_window,
      validation_data = val_window,
      callbacks=[early_stopping])



iteration:   0%|          | 0/3 [00:00<?, ?it/s]

file:   0%|          | 0/61 [00:00<?, ?it/s]

45/45 [==============================] - 14s 317ms/step - loss: 0.0202 - mean_absolute_error: 0.0755 - val_loss: 0.1562 - val_mean_absolute_error: 0.3737


file:   0%|          | 0/61 [00:00<?, ?it/s]

279/279 [==============================] - 99s 354ms/step - loss: 0.0014 - mean_absolute_error: 0.0166 - val_loss: 0.0108 - val_mean_absolute_error: 0.0894


file:   0%|          | 0/61 [00:00<?, ?it/s]

46/46 [==============================] - 18s 391ms/step - loss: 0.0049 - mean_absolute_error: 0.0298 - val_loss: 0.0469 - val_mean_absolute_error: 0.1964


In [ ]:
model_name = f'{INPUT}-{OUTPUT}: {FILE_LOOPS} epochs ({datetime.now()})'

model.save(f'/content/drive/MyDrive/Leren_en_Beslissen/models/{model_name}')

files = {
    'train': [file for file, df in train_list],
    'val': [file for file, df in val_list],
    'test': [file for file, df in test_list]
}

files_file = open(f'/content/drive/MyDrive/Leren_en_Beslissen/models/{model_name}/files.json', "w")
json.dump(files, files_file)
files_file.close()

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Leren_en_Beslissen/models/2016-2016: 3 epochs (2022-01-31 16:48:07.819017)/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Leren_en_Beslissen/models/2016-2016: 3 epochs (2022-01-31 16:48:07.819017)/assets


In [ ]:
for test_file, test_df in tqdm(test_list):
  test_window = to_window(test_df)

  model.evaluate(test_window, return_dict=True)

  0%|          | 0/7 [00:00<?, ?it/s]

89/89 [==============================] - 11s 120ms/step - loss: 0.0013 - mean_absolute_error: 0.0232


In [ ]:
def try_model(window):
  batch = next(iter(window))

  input = tf.stack([batch[0][:, :, i] for i in INPUT_INDICES], axis=-1)
  expected = batch[1]
  prediction = model(batch[0])

  print("input shape:", input.shape)
  print("expected shape:", expected.shape)
  print("predictions shape:", prediction.shape)

  return input, expected, prediction

for test_file, test_df in tqdm(test_list):
  test_window = to_window(test_df)
  
  input, expected, prediction = try_model(test_window)

  # y = input[0, :, :]
  # print(y)
  # x = range(0, INPUT)

  # y = expected[0, :, :]
  # x = range(INPUT, TOTAL)

# def plot(self, model=None, plot_col='averageLeakageRolling', max_subplots=3):
#   inputs, labels = self.example
#   plt.figure(figsize=(12, 8))
#   plot_col_index = self.column_indices[plot_col]
#   max_n = min(max_subplots, len(inputs))
#   for n in range(max_n):
#     plt.subplot(max_n, 1, n+1)
#     plt.ylabel(f'{plot_col} [normed]')
#     plt.plot(self.input_indices, inputs[n, :, plot_col_index],
#              label='Inputs', marker='.', zorder=-10)

#     if self.label_columns:
#       label_col_index = self.label_columns_indices.get(plot_col, None)
#     else:
#       label_col_index = plot_col_index

#     if label_col_index is None:
#       continue

#     plt.scatter(self.label_indices, labels[n, :, label_col_index],
#                 marker='.', edgecolors='k', label='Labels', c='#2ca02c', s=64)
#     if model is not None:
#       predictions = model(inputs)
#       plt.scatter(self.label_indices, predictions[n, :, label_col_index],
#                   marker='.', edgecolors='k', label='Predictions',
#                   c='#ff7f0e', s=64)

#     if n == 0:
#       plt.legend()

#   plt.xlabel('Time Elapsed')

  0%|          | 0/7 [00:00<?, ?it/s]

input shape: (32, 2016, 1)
expected shape: (32, 2016, 1)
predictions shape: (32, 2016, 1)
input shape: (32, 2016, 1)
expected shape: (32, 2016, 1)
predictions shape: (32, 2016, 1)
input shape: (32, 2016, 1)
expected shape: (32, 2016, 1)
predictions shape: (32, 2016, 1)
input shape: (32, 2016, 1)
expected shape: (32, 2016, 1)
predictions shape: (32, 2016, 1)
input shape: (32, 2016, 1)
expected shape: (32, 2016, 1)
predictions shape: (32, 2016, 1)
input shape: (32, 2016, 1)
expected shape: (32, 2016, 1)
predictions shape: (32, 2016, 1)
input shape: (32, 2016, 1)
expected shape: (32, 2016, 1)
predictions shape: (32, 2016, 1)


In [ ]:
# 54/54 [==============================] - 96s 2s/step - loss: 0.0187 - mean_absolute_error: 0.0966 - val_loss: 0.1544 - val_mean_absolute_error: 0.3900
# 448/448 [==============================] - 111s 248ms/step - loss: 0.0617 - mean_absolute_error: 0.1148 - val_loss: 0.0555 - val_mean_absolute_error: 0.2283
# 107/107 [==============================] - 35s 323ms/step - loss: 0.0017 - mean_absolute_error: 0.0275 - val_loss: 0.0854 - val_mean_absolute_error: 0.2860
# 7/7 [==============================] - 8s 1s/step - loss: 0.0382 - mean_absolute_error: 0.1786 - val_loss: 0.0496 - val_mean_absolute_error: 0.2145
# 121/121 [==============================] - 31s 255ms/step - loss: 7.9442e-04 - mean_absolute_error: 0.0163 - val_loss: 0.0139 - val_mean_absolute_error: 0.1068
# 11/11 [==============================] - 4s 387ms/step - loss: 0.0052 - mean_absolute_error: 0.0568 - val_loss: 0.0784 - val_mean_absolute_error: 0.2496
# 67/67 [==============================] - 44s 668ms/step - loss: 0.0196 - mean_absolute_error: 0.0765 - val_loss: 0.0224 - val_mean_absolute_error: 0.0902
# 67/67 [==============================] - 58s 869ms/step - loss: 0.0191 - mean_absolute_error: 0.0764 - val_loss: 0.0235 - val_mean_absolute_error: 0.0917
# 121/121 [==============================] - 71s 587ms/step - loss: 2.2763e-04 - mean_absolute_error: 0.0100 - val_loss: 0.0108 - val_mean_absolute_error: 0.0807
# 195/195 [==============================] - 51s 264ms/step - loss: 0.2359 - mean_absolute_error: 0.3462 - val_loss: 0.0210 - val_mean_absolute_error: 0.1162
# 9/9 [==============================] - 13s 2s/step - loss: 0.0415 - mean_absolute_error: 0.1683 - val_loss: 0.1977 - val_mean_absolute_error: 0.4350
# 35/35 [==============================] - 10s 280ms/step - loss: 0.0163 - mean_absolute_error: 0.1033 - val_loss: 0.2879 - val_mean_absolute_error: 0.5291
# 393/393 [==============================] - 98s 249ms/step - loss: 0.0038 - mean_absolute_error: 0.0211 - val_loss: 0.0617 - val_mean_absolute_error: 0.2206
# 9/9 [==============================] - 13s 2s/step - loss: 0.0140 - mean_absolute_error: 0.0925 - val_loss: 0.0809 - val_mean_absolute_error: 0.2785
# 12/12 [==============================] - 4s 371ms/step - loss: 0.0379 - mean_absolute_error: 0.1634 - val_loss: 0.0077 - val_mean_absolute_error: 0.0793
# 35/35 [==============================] - 50s 1s/step - loss: 0.0554 - mean_absolute_error: 0.1811 - val_loss: 0.0512 - val_mean_absolute_error: 0.1149
# 290/290 [==============================] - 154s 531ms/step - loss: 0.0026 - mean_absolute_error: 0.0390 - val_loss: 0.0985 - val_mean_absolute_error: 0.2865
# 280/280 [==============================] - 110s 393ms/step - loss: 3.8370e-04 - mean_absolute_error: 0.0103 - val_loss: 0.0051 - val_mean_absolute_error: 0.0620
# 249/249 [==============================] - 82s 331ms/step - loss: 0.0068 - mean_absolute_error: 0.0281 - val_loss: 0.3044 - val_mean_absolute_error: 0.5486
# 225/225 [==============================] - 84s 372ms/step - loss: 0.0138 - mean_absolute_error: 0.0263 - val_loss: 0.0172 - val_mean_absolute_error: 0.0745
# 71/71 [==============================] - 100s 1s/step - loss: 0.0055 - mean_absolute_error: 0.0307 - val_loss: 0.0166 - val_mean_absolute_error: 0.0921
# 362/362 [==============================] - 170s 471ms/step - loss: 0.0024 - mean_absolute_error: 0.0373 - val_loss: 0.0290 - val_mean_absolute_error: 0.1463
# 290/290 [==============================] - 81s 281ms/step - loss: 0.0032 - mean_absolute_error: 0.0405 - val_loss: 0.0153 - val_mean_absolute_error: 0.1217
# 46/46 [==============================] - 22s 481ms/step - loss: 0.0111 - mean_absolute_error: 0.0559 - val_loss: 0.0083 - val_mean_absolute_error: 0.0880
# 123/123 [==============================] - 63s 513ms/step - loss: 0.0286 - mean_absolute_error: 0.1294 - val_loss: 0.0045 - val_mean_absolute_error: 0.0584
# 163/163 [==============================] - 85s 522ms/step - loss: 0.0152 - mean_absolute_error: 0.0600 - val_loss: 0.0109 - val_mean_absolute_error: 0.0734
# 45/45 [==============================] - 12s 268ms/step - loss: 5.5034e-04 - mean_absolute_error: 0.0173 - val_loss: 0.0389 - val_mean_absolute_error: 0.1730
# 144/144 [==============================] - 56s 390ms/step - loss: 0.0447 - mean_absolute_error: 0.0379 - val_loss: 0.0042 - val_mean_absolute_error: 0.0442
# 279/279 [==============================] - 70s 252ms/step - loss: 0.0031 - mean_absolute_error: 0.0176 - val_loss: 0.0046 - val_mean_absolute_error: 0.0632
# 1213/1213 [==============================] - 298s 246ms/step - loss: 1.2851e-04 - mean_absolute_error: 0.0076 - val_loss: 0.0136 - val_mean_absolute_error: 0.1046
# 67/67 [==============================] - 78s 1s/step - loss: 0.0262 - mean_absolute_error: 0.1017 - val_loss: 0.0288 - val_mean_absolute_error: 0.1106
# 35/35 [==============================] - 19s 554ms/step - loss: 0.0497 - mean_absolute_error: 0.1721 - val_loss: 0.0149 - val_mean_absolute_error: 0.1003
# 135/135 [==============================] - 54s 404ms/step - loss: 0.0014 - mean_absolute_error: 0.0172 - val_loss: 0.0072 - val_mean_absolute_error: 0.0734
# 225/225 [==============================] - 63s 282ms/step - loss: 0.0123 - mean_absolute_error: 0.0226 - val_loss: 0.0253 - val_mean_absolute_error: 0.0878
# 119/119 [==============================] - 62s 520ms/step - loss: 1.2573 - mean_absolute_error: 0.6066 - val_loss: 0.1263 - val_mean_absolute_error: 0.2911
# 24/24 [==============================] - 16s 692ms/step - loss: 0.0505 - mean_absolute_error: 0.1780 - val_loss: 0.1517 - val_mean_absolute_error: 0.2904
# 11/11 [==============================] - 47s 5s/step - loss: 0.0336 - mean_absolute_error: 0.1298 - val_loss: 0.0769 - val_mean_absolute_error: 0.1897
# 107/107 [==============================] - 30s 285ms/step - loss: 0.0085 - mean_absolute_error: 0.0523 - val_loss: 0.0172 - val_mean_absolute_error: 0.1121
# 225/225 [==============================] - 63s 281ms/step - loss: 0.0151 - mean_absolute_error: 0.0350 - val_loss: 0.0966 - val_mean_absolute_error: 0.2205
# 67/67 [==============================] - 79s 1s/step - loss: 0.0485 - mean_absolute_error: 0.1503 - val_loss: 0.0235 - val_mean_absolute_error: 0.1295
# 7/7 [==============================] - 6s 886ms/step - loss: 0.0097 - mean_absolute_error: 0.0661 - val_loss: 0.0558 - val_mean_absolute_error: 0.2008
# 50/50 [==============================] - 33s 666ms/step - loss: 0.0071 - mean_absolute_error: 0.0381 - val_loss: 0.1621 - val_mean_absolute_error: 0.3731
# 195/195 [==============================] - 48s 245ms/step - loss: 0.2182 - mean_absolute_error: 0.3355 - val_loss: 0.0333 - val_mean_absolute_error: 0.1793
# 24/24 [==============================] - 47s 2s/step - loss: 0.0525 - mean_absolute_error: 0.2021 - val_loss: 0.0179 - val_mean_absolute_error: 0.1084
# 705/705 [==============================] - 210s 298ms/step - loss: 0.0241 - mean_absolute_error: 0.0703 - val_loss: 0.0091 - val_mean_absolute_error: 0.0789
# 225/225 [==============================] - 73s 324ms/step - loss: 0.0167 - mean_absolute_error: 0.0343 - val_loss: 0.1297 - val_mean_absolute_error: 0.3192
# 225/225 [==============================] - 66s 292ms/step - loss: 0.0168 - mean_absolute_error: 0.0351 - val_loss: 0.0278 - val_mean_absolute_error: 0.1270
# 554/554 [==============================] - 177s 320ms/step - loss: 0.0025 - mean_absolute_error: 0.0365 - val_loss: 0.0061 - val_mean_absolute_error: 0.0669
# 50/50 [==============================] - 94s 2s/step - loss: 0.0040 - mean_absolute_error: 0.0358 - val_loss: 0.0131 - val_mean_absolute_error: 0.0851
# 135/135 [==============================] - 65s 482ms/step - loss: 2.2854e-04 - mean_absolute_error: 0.0080 - val_loss: 0.0086 - val_mean_absolute_error: 0.0765
# 554/554 [==============================] - 139s 251ms/step - loss: 0.0024 - mean_absolute_error: 0.0360 - val_loss: 0.0110 - val_mean_absolute_error: 0.0912
# 225/225 [==============================] - 82s 365ms/step - loss: 0.0141 - mean_absolute_error: 0.0235 - val_loss: 0.0092 - val_mean_absolute_error: 0.0702
# 123/123 [==============================] - 74s 608ms/step - loss: 0.0290 - mean_absolute_error: 0.1318 - val_loss: 0.0349 - val_mean_absolute_error: 0.1187
# 46/46 [==============================] - 22s 479ms/step - loss: 0.0101 - mean_absolute_error: 0.0512 - val_loss: 0.0105 - val_mean_absolute_error: 0.0776
# 46/46 [==============================] - 16s 357ms/step - loss: 0.0060 - mean_absolute_error: 0.0381 - val_loss: 0.0048 - val_mean_absolute_error: 0.0333
# 290/290 [==============================] - 151s 520ms/step - loss: 0.0026 - mean_absolute_error: 0.0385 - val_loss: 0.0090 - val_mean_absolute_error: 0.0856
# 135/135 [==============================] - 66s 492ms/step - loss: 0.0013 - mean_absolute_error: 0.0165 - val_loss: 0.0081 - val_mean_absolute_error: 0.0740
# 37/37 [==============================] - 43s 1s/step - loss: 0.0037 - mean_absolute_error: 0.0407 - val_loss: 0.0088 - val_mean_absolute_error: 0.0762
# 393/393 [==============================] - 132s 337ms/step - loss: 5.6578e-04 - mean_absolute_error: 0.0116 - val_loss: 0.0076 - val_mean_absolute_error: 0.0719
# 89/89 [==============================] - 56s 630ms/step - loss: 1.2304e-04 - mean_absolute_error: 0.0076 - val_loss: 0.0076 - val_mean_absolute_error: 0.0720
# 45/45 [==============================] - 14s 317ms/step - loss: 0.0202 - mean_absolute_error: 0.0755 - val_loss: 0.1562 - val_mean_absolute_error: 0.3737

# file: 100%
# 61/61 [1:11:18<00:00, 98.00s/it]

# 290/290 [==============================] - 78s 270ms/step - loss: 0.0023 - mean_absolute_error: 0.0357 - val_loss: 0.0081 - val_mean_absolute_error: 0.0743
# 35/35 [==============================] - 10s 285ms/step - loss: 0.0227 - mean_absolute_error: 0.1210 - val_loss: 0.0053 - val_mean_absolute_error: 0.0681
# 9/9 [==============================] - 6s 739ms/step - loss: 0.1933 - mean_absolute_error: 0.4041 - val_loss: 0.0170 - val_mean_absolute_error: 0.1088
# 225/225 [==============================] - 58s 257ms/step - loss: 0.0142 - mean_absolute_error: 0.0253 - val_loss: 0.0214 - val_mean_absolute_error: 0.1154
# 46/46 [==============================] - 13s 280ms/step - loss: 0.0099 - mean_absolute_error: 0.0586 - val_loss: 0.0050 - val_mean_absolute_error: 0.0398
# 225/225 [==============================] - 57s 253ms/step - loss: 0.0152 - mean_absolute_error: 0.0269 - val_loss: 0.0021 - val_mean_absolute_error: 0.0314
# 50/50 [==============================] - 23s 462ms/step - loss: 0.0033 - mean_absolute_error: 0.0266 - val_loss: 0.2609 - val_mean_absolute_error: 0.4285
# 67/67 [==============================] - 23s 345ms/step - loss: 0.0268 - mean_absolute_error: 0.1023 - val_loss: 0.3016 - val_mean_absolute_error: 0.4778
# 45/45 [==============================] - 30s 680ms/step - loss: 0.0191 - mean_absolute_error: 0.0901 - val_loss: 9.6528e-04 - val_mean_absolute_error: 0.0225
# 12/12 [==============================] - 4s 373ms/step - loss: 0.0204 - mean_absolute_error: 0.1172 - val_loss: 0.0034 - val_mean_absolute_error: 0.0400
# 163/163 [==============================] - 44s 270ms/step - loss: 7.4657e-04 - mean_absolute_error: 0.0175 - val_loss: 0.0148 - val_mean_absolute_error: 0.0985
# 50/50 [==============================] - 23s 469ms/step - loss: 5.8965e-04 - mean_absolute_error: 0.0145 - val_loss: 0.4332 - val_mean_absolute_error: 0.5915
# 54/54 [==============================] - 24s 449ms/step - loss: 0.0319 - mean_absolute_error: 0.0864 - val_loss: 4.4894e-04 - val_mean_absolute_error: 0.0140
# 225/225 [==============================] - 58s 257ms/step - loss: 0.0138 - mean_absolute_error: 0.0201 - val_loss: 0.0483 - val_mean_absolute_error: 0.1934
# 11/11 [==============================] - 44s 4s/step - loss: 0.0071 - mean_absolute_error: 0.0557 - val_loss: 0.0098 - val_mean_absolute_error: 0.0786
# 1213/1213 [==============================] - 312s 257ms/step - loss: 3.1276e-04 - mean_absolute_error: 0.0084 - val_loss: 0.0028 - val_mean_absolute_error: 0.0372
# 554/554 [==============================] - 185s 334ms/step - loss: 0.0023 - mean_absolute_error: 0.0347 - val_loss: 0.0094 - val_mean_absolute_error: 0.0586
# 35/35 [==============================] - 19s 559ms/step - loss: 0.0517 - mean_absolute_error: 0.1713 - val_loss: 0.0121 - val_mean_absolute_error: 0.1073
# 45/45 [==============================] - 17s 372ms/step - loss: 4.4190e-04 - mean_absolute_error: 0.0145 - val_loss: 0.0049 - val_mean_absolute_error: 0.0616
# 225/225 [==============================] - 85s 377ms/step - loss: 0.0145 - mean_absolute_error: 0.0252 - val_loss: 0.0078 - val_mean_absolute_error: 0.0706
# 195/195 [==============================] - 113s 579ms/step - loss: 0.1946 - mean_absolute_error: 0.3268 - val_loss: 0.0139 - val_mean_absolute_error: 0.0974
# 448/448 [==============================] - 119s 266ms/step - loss: 0.0663 - mean_absolute_error: 0.1134 - val_loss: 0.1964 - val_mean_absolute_error: 0.3511
# 89/89 [==============================] - 27s 304ms/step - loss: 4.6914e-04 - mean_absolute_error: 0.0124 - val_loss: 0.0221 - val_mean_absolute_error: 0.1200
# 135/135 [==============================] - 39s 289ms/step - loss: 9.3367e-04 - mean_absolute_error: 0.0135 - val_loss: 0.0049 - val_mean_absolute_error: 0.0600
# 280/280 [==============================] - 77s 274ms/step - loss: 2.4357e-04 - mean_absolute_error: 0.0095 - val_loss: 0.0043 - val_mean_absolute_error: 0.0559
# 46/46 [==============================] - 53s 1s/step - loss: 0.0055 - mean_absolute_error: 0.0388 - val_loss: 0.0013 - val_mean_absolute_error: 0.0309
# 144/144 [==============================] - 78s 545ms/step - loss: 0.0480 - mean_absolute_error: 0.0344 - val_loss: 0.0167 - val_mean_absolute_error: 0.0974
# 249/249 [==============================] - 65s 262ms/step - loss: 0.0012 - mean_absolute_error: 0.0156 - val_loss: 0.0217 - val_mean_absolute_error: 0.1194
# 290/290 [==============================] - 114s 393ms/step - loss: 0.0049 - mean_absolute_error: 0.0422 - val_loss: 0.0106 - val_mean_absolute_error: 0.0841
# 225/225 [==============================] - 103s 457ms/step - loss: 0.0144 - mean_absolute_error: 0.0267 - val_loss: 0.0074 - val_mean_absolute_error: 0.0610
# 107/107 [==============================] - 60s 563ms/step - loss: 0.0071 - mean_absolute_error: 0.0528 - val_loss: 0.0122 - val_mean_absolute_error: 0.0855
# 9/9 [==============================] - 5s 581ms/step - loss: 0.0233 - mean_absolute_error: 0.1156 - val_loss: 0.0460 - val_mean_absolute_error: 0.1774
# 135/135 [==============================] - 44s 326ms/step - loss: 0.0038 - mean_absolute_error: 0.0236 - val_loss: 0.0125 - val_mean_absolute_error: 0.1062
# 71/71 [==============================] - 24s 346ms/step - loss: 5.3286e-05 - mean_absolute_error: 0.0041 - val_loss: 0.0126 - val_mean_absolute_error: 0.1070
# 135/135 [==============================] - 55s 410ms/step - loss: 2.6727e-04 - mean_absolute_error: 0.0090 - val_loss: 0.4940 - val_mean_absolute_error: 0.6884
# 121/121 [==============================] - 37s 306ms/step - loss: 5.6888e-04 - mean_absolute_error: 0.0153 - val_loss: 0.0030 - val_mean_absolute_error: 0.0460
# 37/37 [==============================] - 11s 297ms/step - loss: 0.0025 - mean_absolute_error: 0.0371 - val_loss: 0.0289 - val_mean_absolute_error: 0.1369
# 7/7 [==============================] - 12s 2s/step - loss: 0.0011 - mean_absolute_error: 0.0232 - val_loss: 0.0047 - val_mean_absolute_error: 0.0292
# 393/393 [==============================] - 130s 332ms/step - loss: 1.8755e-04 - mean_absolute_error: 0.0082 - val_loss: 0.0110 - val_mean_absolute_error: 0.0843
# 393/393 [==============================] - 99s 252ms/step - loss: 1.3246e-04 - mean_absolute_error: 0.0070 - val_loss: 3.0222e-04 - val_mean_absolute_error: 0.0137
# 123/123 [==============================] - 41s 332ms/step - loss: 0.0271 - mean_absolute_error: 0.1267 - val_loss: 0.0049 - val_mean_absolute_error: 0.0297
# 67/67 [==============================] - 45s 673ms/step - loss: 0.0197 - mean_absolute_error: 0.0811 - val_loss: 0.0015 - val_mean_absolute_error: 0.0308
# 67/67 [==============================] - 22s 336ms/step - loss: 0.0184 - mean_absolute_error: 0.0713 - val_loss: 0.0067 - val_mean_absolute_error: 0.0305
# 46/46 [==============================] - 20s 436ms/step - loss: 0.0049 - mean_absolute_error: 0.0242 - val_loss: 0.0043 - val_mean_absolute_error: 0.0565
# 705/705 [==============================] - 178s 253ms/step - loss: 0.0219 - mean_absolute_error: 0.0651 - val_loss: 0.0037 - val_mean_absolute_error: 0.0476
# 290/290 [==============================] - 92s 318ms/step - loss: 0.0026 - mean_absolute_error: 0.0373 - val_loss: 0.3085 - val_mean_absolute_error: 0.5256
# 107/107 [==============================] - 30s 277ms/step - loss: 0.0024 - mean_absolute_error: 0.0310 - val_loss: 0.0624 - val_mean_absolute_error: 0.2208
# 35/35 [==============================] - 17s 501ms/step - loss: 0.0331 - mean_absolute_error: 0.1443 - val_loss: 0.0250 - val_mean_absolute_error: 0.1518
# 121/121 [==============================] - 76s 634ms/step - loss: 0.0036 - mean_absolute_error: 0.0333 - val_loss: 0.0133 - val_mean_absolute_error: 0.0805
# 7/7 [==============================] - 7s 1s/step - loss: 0.0025 - mean_absolute_error: 0.0414 - val_loss: 0.0053 - val_mean_absolute_error: 0.0336
# 24/24 [==============================] - 16s 700ms/step - loss: 0.0558 - mean_absolute_error: 0.1960 - val_loss: 0.2079 - val_mean_absolute_error: 0.4022
# 195/195 [==============================] - 50s 258ms/step - loss: 0.2063 - mean_absolute_error: 0.3289 - val_loss: 0.0057 - val_mean_absolute_error: 0.0662
# 67/67 [==============================] - 38s 565ms/step - loss: 0.0200 - mean_absolute_error: 0.0817 - val_loss: 0.3790 - val_mean_absolute_error: 0.6131
# 362/362 [==============================] - 97s 267ms/step - loss: 0.0032 - mean_absolute_error: 0.0405 - val_loss: 0.0132 - val_mean_absolute_error: 0.1021
# 225/225 [==============================] - 58s 257ms/step - loss: 0.0157 - mean_absolute_error: 0.0352 - val_loss: 0.0163 - val_mean_absolute_error: 0.1028
# 24/24 [==============================] - 8s 319ms/step - loss: 0.0505 - mean_absolute_error: 0.1993 - val_loss: 0.0264 - val_mean_absolute_error: 0.1332
# 554/554 [==============================] - 185s 334ms/step - loss: 0.0026 - mean_absolute_error: 0.0372 - val_loss: 0.0243 - val_mean_absolute_error: 0.1377
# 119/119 [==============================] - 58s 492ms/step - loss: 1.1924 - mean_absolute_error: 0.5778 - val_loss: 0.0405 - val_mean_absolute_error: 0.1255
# 123/123 [==============================] - 114s 930ms/step - loss: 0.0248 - mean_absolute_error: 0.1244 - val_loss: 0.0046 - val_mean_absolute_error: 0.0537
# 11/11 [==============================] - 31s 3s/step - loss: 0.0103 - mean_absolute_error: 0.0822 - val_loss: 0.0221 - val_mean_absolute_error: 0.0923
# 279/279 [==============================] - 99s 354ms/step - loss: 0.0014 - mean_absolute_error: 0.0166 - val_loss: 0.0108 - val_mean_absolute_error: 0.0894

# file: 100%
# 61/61 [1:08:34<00:00, 53.46s/it]

# 54/54 [==============================] - 19s 354ms/step - loss: 0.0110 - mean_absolute_error: 0.0647 - val_loss: 0.0122 - val_mean_absolute_error: 0.0976
# 1213/1213 [==============================] - 303s 250ms/step - loss: 1.4330e-04 - mean_absolute_error: 0.0077 - val_loss: 0.0086 - val_mean_absolute_error: 0.0827
# 123/123 [==============================] - 36s 290ms/step - loss: 0.0239 - mean_absolute_error: 0.1206 - val_loss: 0.0075 - val_mean_absolute_error: 0.0542
# 67/67 [==============================] - 18s 272ms/step - loss: 0.0210 - mean_absolute_error: 0.0805 - val_loss: 0.0185 - val_mean_absolute_error: 0.1079
# 107/107 [==============================] - 31s 290ms/step - loss: 0.0068 - mean_absolute_error: 0.0478 - val_loss: 0.0032 - val_mean_absolute_error: 0.0422
# 45/45 [==============================] - 18s 393ms/step - loss: 0.0161 - mean_absolute_error: 0.0640 - val_loss: 0.0059 - val_mean_absolute_error: 0.0622
# 89/89 [==============================] - 31s 345ms/step - loss: 4.6840e-04 - mean_absolute_error: 0.0129 - val_loss: 0.0039 - val_mean_absolute_error: 0.0561
# 290/290 [==============================] - 76s 263ms/step - loss: 0.0024 - mean_absolute_error: 0.0355 - val_loss: 0.0021 - val_mean_absolute_error: 0.0302
# 249/249 [==============================] - 108s 435ms/step - loss: 2.7900e-04 - mean_absolute_error: 0.0108 - val_loss: 0.0064 - val_mean_absolute_error: 0.0684
# 7/7 [==============================] - 3s 393ms/step - loss: 8.3925e-04 - mean_absolute_error: 0.0262 - val_loss: 6.7284e-04 - val_mean_absolute_error: 0.0232
# 225/225 [==============================] - 62s 275ms/step - loss: 0.0141 - mean_absolute_error: 0.0234 - val_loss: 0.0046 - val_mean_absolute_error: 0.0266
# 121/121 [==============================] - 41s 341ms/step - loss: 0.0021 - mean_absolute_error: 0.0272 - val_loss: 0.0026 - val_mean_absolute_error: 0.0352
# 225/225 [==============================] - 139s 622ms/step - loss: 0.0147 - mean_absolute_error: 0.0260 - val_loss: 0.0077 - val_mean_absolute_error: 0.0596
# 225/225 [==============================] - 63s 281ms/step - loss: 0.0160 - mean_absolute_error: 0.0300 - val_loss: 0.0068 - val_mean_absolute_error: 0.0480
# 135/135 [==============================] - 38s 283ms/step - loss: 4.0597e-04 - mean_absolute_error: 0.0113 - val_loss: 0.2302 - val_mean_absolute_error: 0.4561
# 24/24 [==============================] - 11s 476ms/step - loss: 0.0998 - mean_absolute_error: 0.2548 - val_loss: 0.0419 - val_mean_absolute_error: 0.1688
# 67/67 [==============================] - 38s 570ms/step - loss: 0.0184 - mean_absolute_error: 0.0785 - val_loss: 0.0088 - val_mean_absolute_error: 0.0800
# 121/121 [==============================] - 40s 334ms/step - loss: 0.0021 - mean_absolute_error: 0.0283 - val_loss: 0.0055 - val_mean_absolute_error: 0.0362
# 67/67 [==============================] - 19s 278ms/step - loss: 0.0189 - mean_absolute_error: 0.0734 - val_loss: 0.0018 - val_mean_absolute_error: 0.0350
# 11/11 [==============================] - 7s 662ms/step - loss: 8.7429e-04 - mean_absolute_error: 0.0245 - val_loss: 0.0387 - val_mean_absolute_error: 0.1573
# 362/362 [==============================] - 173s 478ms/step - loss: 0.0031 - mean_absolute_error: 0.0403 - val_loss: 0.0032 - val_mean_absolute_error: 0.0440
# 554/554 [==============================] - 142s 256ms/step - loss: 0.0023 - mean_absolute_error: 0.0363 - val_loss: 0.0273 - val_mean_absolute_error: 0.1321
# 9/9 [==============================] - 11s 1s/step - loss: 0.0520 - mean_absolute_error: 0.1933 - val_loss: 0.0378 - val_mean_absolute_error: 0.1924
# 35/35 [==============================] - 14s 406ms/step - loss: 0.0182 - mean_absolute_error: 0.1106 - val_loss: 0.0031 - val_mean_absolute_error: 0.0492
# 11/11 [==============================] - 23s 2s/step - loss: 0.0037 - mean_absolute_error: 0.0541 - val_loss: 0.7215 - val_mean_absolute_error: 0.8458
# 37/37 [==============================] - 50s 1s/step - loss: 0.0018 - mean_absolute_error: 0.0320 - val_loss: 0.0133 - val_mean_absolute_error: 0.0956
# 35/35 [==============================] - 19s 558ms/step - loss: 0.0153 - mean_absolute_error: 0.1014 - val_loss: 0.0387 - val_mean_absolute_error: 0.1949
# 12/12 [==============================] - 8s 731ms/step - loss: 0.0197 - mean_absolute_error: 0.1127 - val_loss: 0.0085 - val_mean_absolute_error: 0.0765
# 554/554 [==============================] - 146s 263ms/step - loss: 0.0024 - mean_absolute_error: 0.0367 - val_loss: 0.0336 - val_mean_absolute_error: 0.1795
# 107/107 [==============================] - 32s 297ms/step - loss: 0.0064 - mean_absolute_error: 0.0467 - val_loss: 0.0027 - val_mean_absolute_error: 0.0386
# 225/225 [==============================] - 138s 617ms/step - loss: 0.0147 - mean_absolute_error: 0.0255 - val_loss: 0.0036 - val_mean_absolute_error: 0.0407
# 46/46 [==============================] - 58s 1s/step - loss: 0.0056 - mean_absolute_error: 0.0383 - val_loss: 0.0169 - val_mean_absolute_error: 0.0879
# 50/50 [==============================] - 46s 933ms/step - loss: 0.0035 - mean_absolute_error: 0.0328 - val_loss: 0.0077 - val_mean_absolute_error: 0.0725
# 50/50 [==============================] - 14s 279ms/step - loss: 1.0489e-04 - mean_absolute_error: 0.0068 - val_loss: 0.0433 - val_mean_absolute_error: 0.1788
# 135/135 [==============================] - 53s 391ms/step - loss: 2.2038e-04 - mean_absolute_error: 0.0079 - val_loss: 0.1013 - val_mean_absolute_error: 0.3116
# 393/393 [==============================] - 139s 354ms/step - loss: 3.0504e-04 - mean_absolute_error: 0.0099 - val_loss: 0.0076 - val_mean_absolute_error: 0.0733
# 144/144 [==============================] - 42s 290ms/step - loss: 0.0462 - mean_absolute_error: 0.0350 - val_loss: 0.0060 - val_mean_absolute_error: 0.0526
# 195/195 [==============================] - 59s 301ms/step - loss: 0.1994 - mean_absolute_error: 0.3299 - val_loss: 0.0859 - val_mean_absolute_error: 0.2526
# 290/290 [==============================] - 74s 257ms/step - loss: 0.0075 - mean_absolute_error: 0.0520 - val_loss: 0.0237 - val_mean_absolute_error: 0.1267
# 24/24 [==============================] - 11s 472ms/step - loss: 0.0551 - mean_absolute_error: 0.1992 - val_loss: 0.0357 - val_mean_absolute_error: 0.1528
# 163/163 [==============================] - 51s 314ms/step - loss: 0.0188 - mean_absolute_error: 0.0630 - val_loss: 0.0063 - val_mean_absolute_error: 0.0482
# 71/71 [==============================] - 37s 522ms/step - loss: 3.8563e-04 - mean_absolute_error: 0.0124 - val_loss: 0.0015 - val_mean_absolute_error: 0.0294
# 290/290 [==============================] - 78s 268ms/step - loss: 0.0028 - mean_absolute_error: 0.0401 - val_loss: 0.0082 - val_mean_absolute_error: 0.0627
# 448/448 [==============================] - 154s 344ms/step - loss: 0.0650 - mean_absolute_error: 0.1132 - val_loss: 0.0059 - val_mean_absolute_error: 0.0665
# 9/9 [==============================] - 11s 1s/step - loss: 0.4550 - mean_absolute_error: 0.6511 - val_loss: 0.0017 - val_mean_absolute_error: 0.0362
# 67/67 [==============================] - 18s 274ms/step - loss: 0.0176 - mean_absolute_error: 0.0819 - val_loss: 0.0551 - val_mean_absolute_error: 0.2042
# 7/7 [==============================] - 3s 401ms/step - loss: 7.3970e-04 - mean_absolute_error: 0.0217 - val_loss: 6.5533e-04 - val_mean_absolute_error: 0.0203
# 280/280 [==============================] - 75s 267ms/step - loss: 6.7690e-04 - mean_absolute_error: 0.0135 - val_loss: 0.0671 - val_mean_absolute_error: 0.2276
# 123/123 [==============================] - 37s 300ms/step - loss: 0.0304 - mean_absolute_error: 0.1340 - val_loss: 0.0032 - val_mean_absolute_error: 0.0476
# 45/45 [==============================] - 94s 2s/step - loss: 0.0094 - mean_absolute_error: 0.0626 - val_loss: 0.0074 - val_mean_absolute_error: 0.0734
# 119/119 [==============================] - 77s 647ms/step - loss: 1.1736 - mean_absolute_error: 0.5547 - val_loss: 0.0077 - val_mean_absolute_error: 0.0748
# 225/225 [==============================] - 119s 531ms/step - loss: 0.0146 - mean_absolute_error: 0.0281 - val_loss: 0.0029 - val_mean_absolute_error: 0.0429
# 705/705 [==============================] - 222s 314ms/step - loss: 0.0212 - mean_absolute_error: 0.0642 - val_loss: 0.0206 - val_mean_absolute_error: 0.1191
# 393/393 [==============================] - 139s 355ms/step - loss: 7.7943e-04 - mean_absolute_error: 0.0142 - val_loss: 0.0013 - val_mean_absolute_error: 0.0322
# 225/225 [==============================] - 62s 278ms/step - loss: 0.0146 - mean_absolute_error: 0.0261 - val_loss: 0.0069 - val_mean_absolute_error: 0.0675
# 135/135 [==============================] - 76s 564ms/step - loss: 5.4121e-04 - mean_absolute_error: 0.0125 - val_loss: 7.0331e-04 - val_mean_absolute_error: 0.0207
# 195/195 [==============================] - 60s 308ms/step - loss: 0.1845 - mean_absolute_error: 0.3241 - val_loss: 0.0052 - val_mean_absolute_error: 0.0386
# 46/46 [==============================] - 53s 1s/step - loss: 0.0052 - mean_absolute_error: 0.0333 - val_loss: 0.0063 - val_mean_absolute_error: 0.0692
# 279/279 [==============================] - 71s 256ms/step - loss: 8.7163e-04 - mean_absolute_error: 0.0141 - val_loss: 0.0183 - val_mean_absolute_error: 0.1085
# 35/35 [==============================] - 37s 1s/step - loss: 0.0153 - mean_absolute_error: 0.0999 - val_loss: 0.0015 - val_mean_absolute_error: 0.0312
# 46/46 [==============================] - 18s 391ms/step - loss: 0.0049 - mean_absolute_error: 0.0298 - val_loss: 0.0469 - val_mean_absolute_error: 0.1964
